In [1]:
import xpress as xp
import pandas as pd
import numpy as np

import re

In [2]:
# Data input
demand = pd.read_excel('Demand_data1.xlsx')

In [3]:
# Create a problem
prob = xp.problem(name='pro')

In [4]:
# Defining the index sets
number_of_periods = 4
number_of_grids = 434
type_of_chargers = 3

number_of_chargers = 78
number_of_pchargers = 325

Periods = range(number_of_periods)
Grids = range(number_of_grids)
Chargers = range(type_of_chargers)


In [5]:
## Define variables 

# calculate the individual grid supply 
isupply = {(g, y): xp.var(lb=0, name='isupply{0}_{1}'.format(g, y))
                for g in Grids for y in Periods}

# final supply amount given the assumption about power sharing between grids
tsupply = {(g, y): xp.var(lb=0, name='tsupply{0}_{1}'.format(g, y))
                for g in Grids for y in Periods}

# demand-supply difference 
diff = {(g, y): xp.var(lb=-50000, name='diff{0}_{1}'.format(g, y))
                for g in Grids for y in Periods}

# store the amound of unmet demand: diff if diff > 0; 0 if diff <= 0
undemand = {(g, y): xp.var(lb=0, name='undemand{0}_{1}'.format(g, y))
                for g in Grids for y in Periods}

#budget: total expenditure = cost of chargers + construction cost
budget = xp.var('budget', lb=0)

prob.addVariable(isupply, tsupply, diff , undemand , budget)

In [6]:
## Adding Stations and points 

#total new points
newpoint = {(g, y): xp.var(vartype=xp.integer,lb=0, name='newpoint{0}_{1}'.format(g, y))
                for g in Grids for y in Periods}

#number of types of new points

newpointty = {(g, c, y): xp.var(vartype=xp.integer, name='newpointty{0}_{1}_{2}'.format(g, c, y))
              for g in Grids for c in Chargers for y in Periods}

#stations = number of stations built at each charging point (max = 20)
newstations = {(g, y): xp.var(vartype=xp.integer,lb=0, name='newstations{0}_{1}'.format(g, y))
                for g in Grids for y in Periods}

#variable for each type of station 
newstationsty = {(g, c, y): xp.var(vartype=xp.integer, name='newstationsty{0}_{1}_{2}'.format(g, c, y))
                for g in Grids for c in Chargers for y in Periods} 

#new_supply from the newly built stations 
newpower = {(g, y): xp.var(name='newpower{0}_{1}'.format(g, y))
                for g in Grids for y in Periods}

#for each pre-existing charging station you can add upto X new stations 
addstations = {(g, y): xp.var(vartype=xp.integer, name='addstations{0}_{1}'.format(g, y))
                for g in Grids for y in Periods}

addstationsty = {(g, c, y): xp.var(vartype=xp.integer, name='addstationsty{0}_{1}_{2}'.format(g, c, y))
                for g in Grids for c in Chargers for y in Periods}

# additional supply for the additional stations: 
addpower = {(g, y): xp.var(name='addpower{0}_{1}'.format(g, y))
                for g in Grids for y in Periods}


prob.addVariable(newpoint, newpointty,
                 newstations, newstationsty,
                 newpower, 
                 addstations, addstationsty,
                 addpower)


In [7]:
# targetdemand

In [8]:
#Problem Parameters
charger_names = ['Slow', 'Fast', 'Rapid']
chargerpower = [2750, 4600, 40250]

chargerprice = [12500, 14000, 52000] ## for slow, fast, rapid chargers
concost = 3000 ## infrastructure construction cost

# 4-year demand matrix for each grid 
targetdemand = demand[['Demand_0','Demand_1','Demand_2','Demand_3']]
targetdemand = targetdemand.to_numpy()

# ##### only use the year 4 demand to test the model
# for g in Grids:
#     targetdemand[g,0:3] = 0
# #####
    
    
#max stations per new point 
maxchargestation = 4
#mas stations added to pre-existing point 
add_lim = 2


#Currentchargers = chargepoints[['Type','grid_number']]
# Current chargers availability:
currentchargers = demand[['Number_of Slow_Charging_Points','Number_of_Fast_Charging_Points','Number_of Rapid_Charging_Points']]
currentchargers = currentchargers.to_numpy() ## convert to array


# Potential Charging points per grid g: 
ppoint_per_grid = demand['Number_of_Potential_Locations']
ppoint_per_grid = ppoint_per_grid.to_numpy()

#Number of pre-existing charging points per grid 
init_points_pgrid = demand['Number_of_Charging_Points']
init_points_pgrid = init_points_pgrid.to_numpy()

#Number of interest points per grid 
interest_points_grid = demand['Number_of_PoI']
interest_points_grid = interest_points_grid.to_numpy()

In [9]:
## current expenditure
cbudget = 0

for g in Grids:
    for c in Chargers:
        #current expenditure = cost of building all chargers already in all grids 
        cbudget += chargerprice[c]*currentchargers[g,c]
    #add the construction cost for each charger that exists
    
cbudget = cbudget + number_of_chargers*concost
    
#print(cbudget) = 1749500

In [10]:
## 'neighbours': matrix of neighbors and itself [343*9, 0 will be filled if num_of_share <9]
## each grid and its neighbours can support charging
neighbors = demand['NEIGHBORS']

neighbours = np.zeros((number_of_grids,9))
for g in Grids:
    nums = neighbors.iloc[g]
    num = re.findall(r'\d+', nums)
    num = list(map(int, num))
    num = np.append(arr = np.array([g+1]), values = num, axis= 0) ## add itself to the start
    #num.append(g+1) ## add itself
    while len(num)<9: ## fill in zeroes for grids with less than 9 neighbors
        #num.append(0)
        num = np.append( arr = np.array([0]), values = num, axis=0)
        
    neighbours[g,]=num
    
neighbours = neighbours.astype(int) ## convert to an integer array

In [11]:
## Grid Power Sharing
## Constraint 1: Filter the neighbours for each grid: Remove the neighbours with no demand 

for g in Grids:
    for i in range(9):
            ne = neighbours[g,i]
            if ne >0:
            ## remove those grids with no demand (based on Year 4 data)
                if (targetdemand[ne-1,3]==0) : 
                    neighbours[ne-1,][neighbours[ne-1,]== (g+1)] = 0 
                    neighbours[g,i] = 0 

In [12]:
## Number of grids that each grid will share its power to 
num_of_share = (neighbours != 0).sum(1)

In [13]:
#current power supply from each grid 
cpower = np.zeros(number_of_grids)
for g in Grids:
    for c in Chargers:
        cpower[g] = cpower[g] + chargerpower[c]*currentchargers[g,c]


In [14]:
## potential charging points


# non-decreasing requirement
prob.addConstraint(newstationsty[g,c,y-1] <= newstationsty[g,c,y] 
                   for c in Chargers for g in Grids for y in Periods if y>0)
prob.addConstraint(newpointty[g,c,y-1] <= newpointty[g,c,y]
                   for c in Chargers for g in Grids for y in Periods if y>0)


#station type must match point type. number of stations per point is limited !!
# since non-decreasing only need to check final year situation

prob.addConstraint(newstationsty[g,c,y] <= maxchargestation*newpointty[g,c,y]
                   for c in Chargers for g in Grids for y in Periods )

# lower bound
prob.addConstraint(newpointty[g,c,y] <= newstationsty[g,c,y] 
                   for c in Chargers for g in Grids for y in Periods)



In [15]:
# the number of new points built per grid per year is the sum of 3 types of charging points
prob.addConstraint(newpoint[g,y] == xp.Sum( newpointty[g,c,y] for c in Chargers) 
                   for g in Grids for y in Periods)


# the total number of new points built per grid limited by the number of potential points per grid
prob.addConstraint(newpoint[g,3] <= ppoint_per_grid[g] for g in Grids )



# prob.addConstraint(newpoint[g,y] <= newstations[g,y]
#                    for g in Grids for y in Periods)

In [16]:
#!! add a lower bound for the number of stations for each built new point


#Power from the new charging points, calculated as the sum of power from each station
prob.addConstraint(newpower[g,y] == xp.Sum( newstationsty[g,c,y]*chargerpower[c] for c in Chargers)
                   for g in Grids for y in Periods )



In [17]:
## additional charging stations 
prob.addConstraint(addstationsty[g,c,y] <= add_lim*currentchargers[g,c]
                   for g in Grids for c in Chargers for y in Periods)
    

In [18]:
#total additional stations is the sum of all the additional stations 
prob.addConstraint(addstations[g,y] == xp.Sum( addstationsty[g,c,y] for c in Chargers )
                   for g in Grids for y in Periods) 

prob.addConstraint(newstations[g,y] == xp.Sum( newstationsty[g,c,y] for c in Chargers )
                   for g in Grids for y in Periods) 

In [19]:
#additional power = sum power from each additional station 
prob.addConstraint(addpower[g,y] == xp.Sum( addstationsty[g,c,y]*chargerpower[c] for c in Chargers )
                   for g in Grids for y in Periods )  


In [20]:
# prob.addConstraint(newstations[g,y-1] <= newstations[g,y] 
#                    for g in Grids for y in Periods if y>0)
# prob.addConstraint(addstations[g,y-1] <= addstations[g,y] 
#                    for g in Grids for y in Periods if y>0)

# prob.addConstraint(newpoint[g,y-1] <= newpoint[g,y] 
#                    for g in Grids for y in Periods if y>0)


In [21]:
## interest points
## each year should satisfy + non-decreasing => year 1 should satisfy 
prob.addConstraint(xp.Sum(init_points_pgrid[ne-1]+addstations[ne-1,0]+newstations[ne-1,0] for ne in neighbours[g,] if (ne >0) and (num_of_share[ne-1]>0) ) >= interest_points_grid[g]
                   for y in Periods for g in Grids if num_of_share[g] >0)



In [22]:
#individual supply from each grid : 
# cpower = current power (from pre-existing stations)
# newpower = power from new stations 
# addpower = power from added stations 
prob.addConstraint( isupply[g,y] == cpower[g] + newpower[g,y] + addpower[g,y]
                   for g in Grids for y in Periods)

# the supply recieved by grid g by all its neighbours 
prob.addConstraint( tsupply[g,y] == xp.Sum(isupply[ne-1,y]/num_of_share[ne-1] 
                                         for ne in neighbours[g,] if (ne >0) and (num_of_share[ne-1]>0))
                    for g in Grids for y in Periods)


In [23]:
## City Centre: G215 + neighbouring grids
## sensitivity analysis: change the bound, 

## Special requirement: 1. The number of charging stations must not exceed some bound
## 2. The total number of charging stations must not exceed ... (after Fatima's model)

citycentre = [184,185,186,198,199,200,201,213,214,215,229]

prob.addConstraint(xp.Sum(init_points_pgrid[cg-1]+addstations[cg-1,3]+newstations[cg-1,3] for cg in citycentre) <= 30
                  for y in Periods)

In [24]:
# Deficit = difference between the demand and supply of grid g 
prob.addConstraint( diff[g,y] == targetdemand[g,y] - tsupply[g,y] 
                   for g in Grids for y in Periods)



In [25]:
## Diversification
## Proportion of rapid chargers must not exceed 20%
## xp.Sum(currentchargers[g,2] for g in Grids) = 13

prob.addConstraint( xp.Sum(currentchargers[j,2] + newstationsty[j,2,3] + addstationsty[j,2,3] 
                           for j in Grids ) <= 
                    0.2*(number_of_chargers+ xp.Sum( newstationsty[g,c,3] + addstationsty[g,c,3] 
                                                    for c in Chargers for g in Grids )) )



In [26]:
## expenditure !!!
# prob.addConstraint( budget == xp.Sum(chargerprice[c]*(newstationsty[g,c,y]+addstationsty[g,c,y]) 
#                                      for g in Grids for c in Chargers for y in Periods)
#                    + concost*(xp.Sum(newpoint[g,y] for g in Grids for y in Periods)) )


prob.addConstraint( budget == xp.Sum(chargerprice[c]*(newstationsty[g,c,3]+addstationsty[g,c,3]) 
                                     for g in Grids for c in Chargers )
                   + concost*(xp.Sum(newpoint[g,3] for g in Grids)) )


In [27]:
## The limita
prob.addConstraint( budget <= 10 * cbudget)

In [28]:
## unmet demand 
prob.addConstraint( diff[g,y] <= undemand[g,y] for g in Grids for y in Periods)

prob.addConstraint( xp.Sum(undemand[g,y] for g in Grids for y in Periods) <= 0)

In [29]:
#minimize the unmet demand 

prob.setObjective( budget, sense = xp.minimize)

In [ ]:
#prob.write("pro","lp")
prob.solve()

FICO Xpress v8.13.7, Hyper, solve started 0:05:52, Nov 28, 2022
Heap usage: 19MB (peak 19MB, 13MB system)
Minimizing MILP pro using up to 4 threads, with these control settings:
OUTPUTLOG = 1
Original problem has:
     39914 rows        31249 cols       105362 elements     20832 globals
Presolved problem has:
      4721 rows         2808 cols        13345 elements      2427 globals
LP relaxation tightened
Presolve finished in 0 seconds
Heap usage: 24MB (peak 46MB, 13MB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.11e-01,  5.20e+04] / [ 6.36e-03,  1.36e+01]
  RHS and bounds [min,max] : [ 1.00e+00,  1.75e+07] / [ 3.26e-02,  7.87e+03]
  Objective      [min,max] : [ 1.00e+00,  1.00e+00] / [ 8.19e+03,  8.19e+03]
Autoscaling applied Curtis-Reid scaling

Symmetric problem: generators: 2, support set: 18
 Number of orbits: 6, largest orbit: 3
 Row orbits: 6, row support: 18
Will try to keep branch and bound tree memory u

     718  2537000.000  2497102.128     23    491     58    1.57%      78      8
     820  2537000.000  2497102.128     23    588     64    1.57%      93      9
     926  2537000.000  2497761.338     23    636     36    1.55%     170      9
R   1010  2532000.000  2497761.338     24    715     17    1.35%       0      9
    1029  2532000.000  2497761.338     24    715     45    1.35%      73      9
    1169  2532000.000  2499087.953     24    668      1    1.30%       1      9
    1270  2532000.000  2499087.953     24    696     64    1.30%      60     10
    1373  2532000.000  2499087.953     24    836     37    1.30%     156     10
    1475  2532000.000  2499087.953     24    890     50    1.30%     120     10
    1576  2532000.000  2499087.953     24    902     63    1.30%      62     10
    1681  2532000.000  2499097.002     24   1036     54    1.30%     109     10
R   1681  2529000.000  2499097.002     25   1036     18    1.18%       0     11
    1799  2529000.000  2499097.002     2

  16  K   2529000.000  2504550.240     25     55    167    0.97%     144     25
  17  K   2529000.000  2504550.240     25     55     59    0.97%     123     25
  18  K   2529000.000  2504550.240     25     51     34    0.97%     133     25
  19  K   2529000.000  2504550.240     25     45     76    0.97%     140     25
  20  K   2529000.000  2504550.240     25     20     30    0.97%     147     25
  21  K   2529000.000  2504550.240     25     58     29    0.97%     129     25
  22  K   2529000.000  2504550.240     25    148     43    0.97%     104     25
  23  K   2529000.000  2504550.240     25    130    127    0.97%     152     25
  24  K   2529000.000  2504550.240     25     92    135    0.97%     135     25
  25  K   2529000.000  2504550.240     25     63     78    0.97%     133     25
  26  K   2529000.000  2504550.240     25     21     48    0.97%     117     26
  27  K   2529000.000  2504550.240     25      1     18    0.97%     119     26
  28  K   2529000.000  2504550.240     2

   13192  2529000.000  2504550.240     25   3488     21    0.97%     151     44
   13300  2529000.000  2504550.240     25   3543     60    0.97%      69     44
   13403  2529000.000  2504550.240     25   3636     32    0.97%     133     44
   13510  2529000.000  2504550.240     25   3670     29    0.97%     170     45
   13615  2529000.000  2504550.240     25   3730     32    0.97%     157     45
   13717  2529000.000  2504550.240     25   3776     46    0.97%      96     45
Elapsed time (sec): 45, estimated tree completion: 0.02310
Heap usage: 168MB (peak 191MB, 15MB system)
B&B tree size: 33MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
   13817  2529000.000  2504550.240     25   3830     16    0.97%     173     45
   13918  2529000.000  2504550.240     25   3910     24    0.97%     151     46
   14023  2529000.000  2504550.240     25   3941     30    0.97%     141     46
   14128  2529000.000  2504550.240     25   4019     33    0.97%     

   66763  2529000.000  2504550.240     25  34111     43    0.97%      95    148
   67767  2529000.000  2504550.240     25  34664     23    0.97%     117    149
   68771  2529000.000  2504550.240     25  35173     33    0.97%     141    151
   69771  2529000.000  2504550.240     25  35752     16    0.97%     203    153
   70779  2529000.000  2504550.240     25  36313     30    0.97%     145    155
   71780  2529000.000  2504550.240     25  36915     16    0.97%     189    157
   72792  2529000.000  2504550.240     25  37481     55    0.97%     101    160
   73800  2529000.000  2504550.240     25  38084     39    0.97%     131    162
   74805  2529000.000  2504550.240     25  38673     29    0.97%     148    164
   75807  2529000.000  2504550.240     25  39194     32    0.97%     115    166
   76808  2529000.000  2504550.240     25  39795     54    0.97%      58    168
   77808  2529000.000  2504550.240     25  40404     21    0.97%     135    169
   78817  2529000.000  2504550.240     2

In [ ]:
#prob.iisfirst 

In [ ]:
prob.getSolution(budget)

In [ ]:
# diff_df = pd.DataFrame(data = prob.getSolution(diff), index = Grids, columns=Periods )

In [ ]:
a = prob.getSolution(newstations)

{(g,y):x for (g,y),x in a.items() if x!=0 and y==1}

In [ ]:
{(g,y):x for (g,y),x in prob.getSolution(addstations).items() if x!=0 and y==1}

In [ ]:
{(g,y):x for (g,y),x in prob.getSolution(newpoint).items() if x!=0 and y==1}

In [ ]:
len({(g,y):x for (g,y),x in prob.getSolution(newpoint).items() if x!=0 })

In [ ]:
numcen = 0
for i in citycentre:
    numcen += newstations_df.iloc[i-1,0] + addstations_df.iloc[i-1,0] + init_points_pgrid[i-1]
    
numcen

In [ ]:
## Output

# undemand_df = pd.DataFrame(data = prob.getSolution(undemand), index = Grids, columns=Periods)

# undemand_df.loc[(undemand_df!=0).any(1) ]

In [ ]:
## see the grids with 
# tsupply_df = pd.DataFrame(data = prob.getSolution(tsupply), index = Grids, columns=Periods)

# tsupply_df.loc[(tsupply_df!=0).any(1) ]